## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,19.13,78,100,40.67,overcast clouds
1,1,Bilma,NE,18.6853,12.9164,57.99,22,3,6.67,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,57.22,62,0,8.05,clear sky
3,3,Woodward,US,36.4337,-99.3904,36.63,51,0,10.36,clear sky
4,4,Praia Da Vitoria,PT,38.7333,-27.0667,57.76,94,40,11.79,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for travel?"))
max_temp = float(input("What is the maximum temperature you would like for travel?"))


What is the minimum temperature you would like for travel?85
What is the maximum temperature you would like for travel?95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_city_data_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]< max_temp)]
new_city_data_df 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
55,55,Parrita,CR,9.5201,-84.3225,92.97,74,17,5.21,few clouds
64,64,Champerico,GT,14.3000,-91.9167,93.74,61,6,12.66,clear sky
103,103,Kahului,US,20.8947,-156.4700,85.26,73,0,8.99,clear sky
184,184,Ciudad Bolivar,VE,8.1222,-63.5497,88.16,36,30,8.14,scattered clouds
212,212,Guayaquil,EC,-2.1667,-79.9000,87.76,58,75,5.75,broken clouds
216,216,Boa Vista,BR,2.8197,-60.6733,87.78,48,40,10.36,scattered clouds
362,362,Poya,NC,-21.3500,165.1500,85.23,74,94,4.68,overcast clouds
439,439,San Juan Del Sur,NI,11.2529,-85.8705,88.02,51,20,4.61,few clouds
544,544,Pinillos,CO,8.9193,-74.4677,85.80,59,42,4.65,scattered clouds
720,720,Karratha,AU,-20.7377,116.8463,85.17,67,98,7.83,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
new_city_data_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_city_data_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 55 to 720
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              10 non-null     int64  
 1   City                 10 non-null     object 
 2   Country              10 non-null     object 
 3   Lat                  10 non-null     float64
 4   Lng                  10 non-null     float64
 5   Max Temp             10 non-null     float64
 6   Humidity             10 non-null     int64  
 7   Cloudiness           10 non-null     int64  
 8   Wind Speed           10 non-null     float64
 9   Current Description  10 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 880.0+ bytes


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 55 to 720
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 10 non-null     object 
 1   Country              10 non-null     object 
 2   Max Temp             10 non-null     float64
 3   Current Description  10 non-null     object 
 4   Lat                  10 non-null     float64
 5   Lng                  10 non-null     float64
 6   Hotel Name           10 non-null     object 
dtypes: float64(3), object(4)
memory usage: 640.0+ bytes


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params ["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels["results"][0]["name"]
    except:
         print("Hotel not found ... skipping")
        
hotel_df

Hotel not found ... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
55,Parrita,CR,92.97,few clouds,9.5201,-84.3225,Beso del Viento
64,Champerico,GT,93.74,clear sky,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
103,Kahului,US,85.26,clear sky,20.8947,-156.4700,Maui Seaside Hotel
184,Ciudad Bolivar,VE,88.16,scattered clouds,8.1222,-63.5497,Hotel Edi
212,Guayaquil,EC,87.76,broken clouds,-2.1667,-79.9000,HMHotel
216,Boa Vista,BR,87.78,scattered clouds,2.8197,-60.6733,Aipana Plaza Hotel
362,Poya,NC,85.23,overcast clouds,-21.3500,165.1500,
439,San Juan Del Sur,NI,88.02,few clouds,11.2529,-85.8705,Nuestra Casa
544,Pinillos,CO,85.80,scattered clouds,8.9193,-74.4677,HOTEL CASA BLANCA
720,Karratha,AU,85.17,overcast clouds,-20.7377,116.8463,ibis Styles Karratha


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 55 to 720
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 10 non-null     object 
 1   Country              10 non-null     object 
 2   Max Temp             10 non-null     float64
 3   Current Description  10 non-null     object 
 4   Lat                  10 non-null     float64
 5   Lng                  10 non-null     float64
 6   Hotel Name           10 non-null     object 
dtypes: float64(3), object(4)
memory usage: 640.0+ bytes


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


In [12]:
# 11a. Add a marker layer for each city to the map.
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))